# Demos: Lecture 15

In [3]:
import pennylane as qml
from pennylane import numpy as np

## Demo 1: Order finding

In [4]:
from lecture15_helpers import *

In [5]:
N = 15

num_estimation_wires = int(np.log2(N) + 1)
num_target_wires = num_estimation_wires
num_work_wires = num_target_wires + 2
total_wires = num_estimation_wires + num_target_wires + num_work_wires

estimation_wires = range(num_estimation_wires)
target_wires = range(num_estimation_wires, num_estimation_wires + num_target_wires)
work_wires = range(target_wires[-1] + 1,  target_wires[-1] + 1 + num_work_wires)

dev = qml.device('default.qubit', wires=total_wires, shots=100)

In [6]:
def hadamard_transform(wires):
    for wire in wires:
        qml.Hadamard(wires=wire)

@qml.qnode(dev)
def find_order(N, a):
    # Prepare the target register in some state
    qml.PauliX(wires=target_wires[-1])

    # Run QPE
    hadamard_transform(estimation_wires)
     
    for est_wire in range(num_estimation_wires):
        power_of_2 = num_estimation_wires - est_wire - 1
        multiply_by = pow(a, 2 ** power_of_2) % N
        qml.ctrl(qml.Multiplier, control=est_wire)(multiply_by, target_wires, mod=N, work_wires=work_wires)

    qml.adjoint(qml.QFT)(estimation_wires)

    return qml.sample(wires=estimation_wires)

In [7]:
samples = find_order(N, 2)
phases = [fractional_binary_to_float(sample) for sample in samples]
est_r = [phase_to_order(phase, N) for phase in phases]

print(est_r)

[4, 1, 4, 1, 4, 1, 2, 1, 1, 2, 4, 1, 4, 4, 4, 1, 1, 1, 4, 2, 4, 4, 4, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 1, 2, 4, 4, 2, 2, 1, 4, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 4, 4, 4, 2, 1, 4, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 2, 4, 4, 4, 1, 4, 4, 4, 4, 4, 1, 4, 1, 1, 4, 4, 4, 1, 2, 4, 1, 4, 4]


In [8]:
print(qml.draw(find_order)(N, 2))

 0: ──H─╭●──────────────────────────────────────────────╭QFT†─┤ ╭Sample
 1: ──H─│───────────╭●──────────────────────────────────├QFT†─┤ ├Sample
 2: ──H─│───────────│───────────╭●──────────────────────├QFT†─┤ ├Sample
 3: ──H─│───────────│───────────│───────────╭●──────────╰QFT†─┤ ╰Sample
 4: ────├Multiplier─├Multiplier─├Multiplier─├Multiplier───────┤        
 5: ────├Multiplier─├Multiplier─├Multiplier─├Multiplier───────┤        
 6: ────├Multiplier─├Multiplier─├Multiplier─├Multiplier───────┤        
 7: ──X─├Multiplier─├Multiplier─├Multiplier─├Multiplier───────┤        
 8: ────├Multiplier─├Multiplier─├Multiplier─├Multiplier───────┤        
 9: ────├Multiplier─├Multiplier─├Multiplier─├Multiplier───────┤        
10: ────├Multiplier─├Multiplier─├Multiplier─├Multiplier───────┤        
11: ────├Multiplier─├Multiplier─├Multiplier─├Multiplier───────┤        
12: ────├Multiplier─├Multiplier─├Multiplier─├Multiplier───────┤        
13: ────╰Multiplier─╰Multiplier─╰Multiplier─╰Multiplier───────┤ 

## Demo 2: Shor's algorithm

<img src="fig/shor-flowchart.jpeg" width="300px">

For a more detailed implementation, including all the operations involved in modular exponentiation, you can check out my [work-in-progress PennyLane demo](https://qml-build-previews.pennylane.ai/pull_request_build_preview/1010/qml/demos/tutorial_shor_catalyst) (link is to a PR build; not final version).

In [9]:
def shors_algorithm(N):
    p, q = 0, 0

    while p * q != N:
        a = int(np.random.choice(np.arange(2, N - 1)))
        print(f"Trying a = {a}\n") 

        if np.gcd(N, a) != 1:
            print(f"a = {a} has a common factor with N\n")
            continue
            # p = np.gcd(N, a)
            # return p, N // p

        samples = find_order(N, a)
        phases = [fractional_binary_to_float(sample) for sample in samples]
        est_r = [phase_to_order(phase, N) for phase in phases]

        successes = 0
 
        for r in est_r:
            if r % 2 == 0:
                guess_square_root = (a ** (r // 2)) % N
    
                if guess_square_root not in [1, N - 1]:
                    p = np.gcd(N, guess_square_root - 1)
                    q = np.gcd(N, guess_square_root + 1)
                    if p * q == N:
                        successes += 1
        if successes > 0:
            break
        
    if successes > 0:
        print(f"Success probability = {successes/dev.shots.total_shots}")
        
    else:
        print(f"Solution not found after {dev.shots.total_shots} shots.")
        
    return p, q

In [10]:
p, q = shors_algorithm(N)
print(f"Found solution N = {N} = {p} x {q}")

Trying a = 8

Success probability = 0.55
Found solution N = 15 = 3 x 5
